Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_failure_upper_bound">_failure_upper_bound</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import a, b, e, f, i, l, x, defaults, Lambda
from proveit.logic import Equals, InSet, NotEquals
from proveit.numbers import (zero, one, two, four, Add, Exp, frac, greater,
                             greater_eq,Less, Mult, NaturalPos, Neg, Real, subtract)
from proveit.numbers.functions import MonDecFuncs
# import QPE common expressions
from proveit.physics.quantum.QPE import (_alpha_l, _alpha_l_sqrd, _t, _delta,
    _two_pow_t, _full_domain, _neg_domain, _pos_domain,
    _diff_l_scaled_delta_floor)
# import QPE axioms
from proveit.physics.quantum.QPE import _t_in_natural_pos, _mod_add_def
# import QPE theorems
from proveit.physics.quantum.QPE import (
    _b_floor, _best_floor_is_int,
    _pos_domain_in_full_domain_sans_zero, _neg_domain_in_full_domain_sans_zero,
    _pos_domain_within_integer, _neg_domain_within_integer,
    _two_pow_t_minus_one_is_nat_pos, _phase_is_real, 
    _delta_b_is_real, _scaled_delta_b_floor_in_interval,
    _scaled_delta_b_is_zero_or_non_int )
from proveit.physics.quantum.QPE.phase_est_ops import Psuccess, Pfail, ModAdd

In [ ]:
%proving _failure_upper_bound

In [ ]:
defaults.assumptions = _failure_upper_bound.conditions

In [ ]:
# useful to name this sub-expression; checking to see if we can omit
# _scaled_delta = Mult(_two_pow_t, _delta)

In [ ]:
# temporarily seeing if this is useful 20221014
from proveit.physics.quantum.QPE import _delta_b_floor
_scaled_delta_b = Mult(_two_pow_t, _delta_b_floor)

### (1) First major step(s) rely on the `_failure_upper_bound_lemma`

In [ ]:
from proveit.physics.quantum.QPE import _failure_upper_bound_lemma
_failure_upper_bound_lemma

In [ ]:
# this seems to be replacing the 2^{t-1} with some alternate form?
_failure_upper_bound_lemma_inst = (
    _failure_upper_bound_lemma.instantiate(preserve_expr=Neg(Add(e, one))))

### (2) Establish some convenient labels for important expressions

In [ ]:
first_sum = _failure_upper_bound_lemma_inst.rhs.operands[1].operands[0]

In [ ]:
second_sum = _failure_upper_bound_lemma_inst.rhs.operands[1].operands[1]

In [ ]:
the_summand = first_sum.summand

In [ ]:
first_sum_conditions = first_sum.conditions[0]

In [ ]:
second_sum_conditions = second_sum.conditions[0]

In [ ]:
e_conditions = _failure_upper_bound.conditions[0]

### Upper bound each of the summations in `fail_ineq_lemma_inst` via …

### (3) Some Initial Bounding of the Summand Denominator

In [ ]:
_scaled_delta_b_floor_in_interval

In [ ]:
_delta_b_is_real.instantiate({b:_b_floor})

In [ ]:
# Automatic incidental derivation, but want to label the result for later use
scaled_delta_lower_bound = _scaled_delta_b_floor_in_interval.derive_element_lower_bound()

In [ ]:
# Automatic incidental derivation, but want to label the result for later use
scaled_delta_upper_bound = _scaled_delta_b_floor_in_interval.derive_element_upper_bound()

In [ ]:
neg_scaled_delta_lower_bound = scaled_delta_upper_bound.left_mult_both_sides(Neg(one))

In [ ]:
neg_scaled_delta_upper_bound = scaled_delta_lower_bound.left_mult_both_sides(Neg(one))

In [ ]:
_diff_l_scaled_delta_floor.deduce_bound(neg_scaled_delta_lower_bound,
        assumptions=defaults.assumptions + (first_sum_conditions,))

In [ ]:
# For the 1st Summation
_diff_l_scaled_delta_upper_bound = _diff_l_scaled_delta_floor.deduce_bound(
        neg_scaled_delta_upper_bound.reversed(),
        assumptions=defaults.assumptions + (first_sum_conditions,))

In [ ]:
# For the 2nd Summation
_diff_l_scaled_delta_upper_bound_2 = _diff_l_scaled_delta_floor.deduce_bound(
        neg_scaled_delta_lower_bound,
        assumptions=defaults.assumptions + (second_sum_conditions,))

Also need to demonstrate that we have:

(1) For the 1st Summation: $\ell < 0$ and $\ell^2 \in \mathbb{R}^{+}$

(2) For the 2nd Summation: $\ell - 1 > 0$ and $\ell^2 \in \mathbb{R}^{+}$

In [ ]:
e_greater_eq_one = greater_eq(e, one).prove()

In [ ]:
e_plus_one_greater_eq_two = e_greater_eq_one.right_add_both_sides(one)

In [ ]:
neg_e_plus_one_less_eq_neg_two = Neg(Add(e, one)).bound_via_operand_bound(e_plus_one_greater_eq_two)

In [ ]:
# For 1st summation
ell_upper_bound = first_sum_conditions.derive_element_upper_bound(
    assumptions=defaults.assumptions + (first_sum_conditions,))

In [ ]:
# For 2nd Summation
ell_lower_bound_02 = second_sum_conditions.derive_element_lower_bound(
        assumptions = defaults.assumptions + (second_sum_conditions, ))

In [ ]:
# For 1st Summation
ell_upper_bound_neg_two = ell_upper_bound.apply_transitivity(neg_e_plus_one_less_eq_neg_two)

In [ ]:
# For Second Summation
ell_lower_bound_two_2 = ell_lower_bound_02.apply_transitivity(e_plus_one_greater_eq_two)

In [ ]:
# OMIT
# neg_two_less_zero = Less(Neg(two), zero).prove()

In [ ]:
# OMIT
# For 1st Summation
# ell_less_zero = ell_upper_bound_neg_two.apply_transitivity(neg_two_less_zero)

In [ ]:
# OMIT
# For 2nd Summation
# ell_minus_one_greater_one_in_sum_02 = ell_lower_bound_two_2.right_add_both_sides(Neg(one))

In [ ]:
second_sum_conditions

In [ ]:
# Still seems to be needed!
# needed for next step
greater(subtract(l, one), zero).prove(
    assumptions=defaults.assumptions + (second_sum_conditions,))

In [ ]:
# Checking on this now for omission.
# For 2nd Summation (actually needed this explicitly for later)
from proveit.numbers import RealPos
InSet(Exp(subtract(l, one), two), RealPos).prove(
    assumptions=defaults.assumptions + (second_sum_conditions,))

In [ ]:
temp_bound_03 = ell_upper_bound_neg_two.right_add_both_sides(
    neg_scaled_delta_lower_bound.lhs)

In [ ]:
ell_lower_bound_two_2.right_add_both_sides(neg_scaled_delta_lower_bound.lhs)

In [ ]:
display(scaled_delta_lower_bound)
display(scaled_delta_upper_bound)

In [ ]:
scaled_delta_plus_2_lower_bound = scaled_delta_lower_bound.left_add_both_sides(two)

In [ ]:
temp_bound_06 = scaled_delta_plus_2_lower_bound.left_mult_both_sides(Neg(one)).with_styles(direction='normal')

In [ ]:
temp_bound_07 = temp_bound_03.apply_transitivity(temp_bound_06)

In [ ]:
# OMIT
# For 2nd summation
# neg_scaled_delta_upper_bound = scaled_delta_upper_bound.left_mult_both_sides(Neg(one))

In [ ]:
# OMIT
# For 2nd summation
# two_minus_scaled_delta_upper_bound = neg_scaled_delta_upper_bound.left_add_both_sides(two)

In [ ]:
# OMIT? Trying this again and keeping the one bel;ow as a substitute
# greater(two_minus_scaled_delta_upper_bound.lhs, zero).prove()
# greater(subtract(two, _scaled_delta_b), zero).prove()

In [ ]:
# trying this instead of the one above
# does it later suffice for intantiation when we need to know the diff ≠ 0?
# greater(subtract(one, _scaled_delta_b), zero).prove()

In [ ]:
# trying this instead of the one above
# might need both this one and the one above
# the one above is apparently not sufficient for later instantiation if we need ≠ 0
# NotEquals(subtract(one, _scaled_delta_b), zero).prove()

In [ ]:
# good grief, we seem to need something like this:
# NotEquals(subtract(l, _scaled_delta_b), zero).prove()

#### Bound on the First Summand and First Sum
For the summand in the first sum, then, we can find an $\ell$-dependent upper bound.

In [ ]:
# OMIT
# A reminder of the_summand
the_summand

In [ ]:
# Actually we seem to still need this one?
Less(the_summand.denominator.base, zero).prove(assumptions=defaults.assumptions + (first_sum_conditions,))

In [ ]:
# OMIT
# NotEquals(the_summand.denominator, zero).prove(assumptions=defaults.assumptions + (first_sum_conditions,))

In [ ]:
# OMIT
# greater(the_summand.denominator, zero).prove(
#     assumptions=defaults.assumptions + (first_sum_conditions,))

In [ ]:
# OMIT
# first_summand_bound_02 = the_summand.deduce_bound(_diff_l_scaled_delta_upper_bound,
#         assumptions=defaults.assumptions + (first_sum_conditions,))

In [ ]:
# OMIT
# scaled_delta_lower_bound

In [ ]:
first_summand_bound_02 = the_summand.deduce_bound(scaled_delta_lower_bound.reversed(),
        assumptions=defaults.assumptions + (first_sum_conditions,))

In [ ]:
first_summand_bound_generalized = first_summand_bound_02.generalize((l), conditions=[first_sum_conditions])

In [ ]:
first_sum_bound = first_sum.deduce_bound(first_summand_bound_generalized)

#### Bound on the Second Summand and Second Sum

In [ ]:
second_summand_bound_02 = the_summand.deduce_bound(_diff_l_scaled_delta_upper_bound_2,
        assumptions=defaults.assumptions + (second_sum_conditions,))

In [ ]:
second_summand_bound_02_relaxed = second_summand_bound_02.derive_relaxed()

In [ ]:
second_summand_bound_generalized = second_summand_bound_02.generalize([l], conditions=[second_sum_conditions])

In [ ]:
second_summand_bound_relaxed_generalized = second_summand_bound_02_relaxed.generalize([l], conditions=[second_sum_conditions])

In [ ]:
second_sum_bound = second_sum.deduce_bound(second_summand_bound_relaxed_generalized)

Now we're ready to invoke our `deduce_bound()` method utilizing the bound on the first summation and utilizing the bound on the second summation. The following three steps correspond to the transition from (5.30) to (5.31) in Nielsen & Chuang (pg 224):

In [ ]:
_failure_upper_bound_lemma_inst_bound_01 = _failure_upper_bound_lemma_inst.rhs.deduce_bound(first_sum_bound)

In [ ]:
_failure_upper_bound_lemma_inst_bound_02 = _failure_upper_bound_lemma_inst_bound_01.rhs.deduce_bound(second_sum_bound)

In [ ]:
_failure_upper_bound_lemma_inst_bound_03 = (
    _failure_upper_bound_lemma_inst_bound_01.apply_transitivity(_failure_upper_bound_lemma_inst_bound_02))

Next, shift the index of the second summation to create identical summands in each of the summations:

In [ ]:
_failure_upper_bound_lemma_inst_bound_04 = (
    _failure_upper_bound_lemma_inst_bound_03.inner_expr().rhs.operands[1].
    operands[1].shift(Neg(one)))

Then negate and flip the order of indices on the first summation (possible because the summand is an even function).

In [ ]:
# ACTUALLY, this theorem is FALSE, so this is quite annoying
# This works for something like x^2 or even 1/x^2 (as we have),
# but not more generally for something like (x-1)^2.
# In other words, we need f(x) \in EvenFuncs.
from proveit.numbers.summation import even_fxn_sum
even_fxn_sum

In [ ]:
_a_sub, _b_sub, _x_sub, _f_sub = (
    first_sum_conditions.domain.lower_bound,
    first_sum_conditions.domain.upper_bound,
    l,
    Lambda(l, frac(one, l)))

In [ ]:
even_fxn_sum_inst = even_fxn_sum.instantiate({a: _a_sub, b: _b_sub, x: _x_sub, f: _f_sub})

In [ ]:
first_sum_conditions_reversed = even_fxn_sum_inst.rhs.conditions[0]

In [ ]:
fxn_sqrd_eq_one_over_sqr_01 = Exp(frac(one, l), two).distribution(assumptions=defaults.assumptions+(first_sum_conditions,))

In [ ]:
# under the reversed index conditions, need to know that ell is still real?
InSet(l, Real).prove(assumptions=defaults.assumptions+(first_sum_conditions_reversed,))

In [ ]:
# and under the reversed index conditions, need to know the substitution works
fxn_sqrd_eq_one_over_sqr_02 = Exp(frac(one, l), two).distribution(assumptions=defaults.assumptions+(first_sum_conditions_reversed,))

In [ ]:
even_fxn_sum_inst_sub_lhs = even_fxn_sum_inst.inner_expr().lhs.summand.substitute(
    fxn_sqrd_eq_one_over_sqr_01,
    assumptions=defaults.assumptions+(first_sum_conditions,))

In [ ]:
even_fxn_sum_inst_sub_lhs_rhs = even_fxn_sum_inst_sub_lhs.inner_expr().rhs.summand.substitute(
    fxn_sqrd_eq_one_over_sqr_02,
    assumptions=defaults.assumptions+(first_sum_conditions_reversed,))

In [ ]:
# recall the state of our bound thus far
_failure_upper_bound_lemma_inst_bound_04

In [ ]:
_failure_upper_bound_lemma_inst_bound_05 = (
    _failure_upper_bound_lemma_inst_bound_04.inner_expr().rhs.operands[1].
    operands[0].substitute(even_fxn_sum_inst_sub_lhs_rhs))

To split the first term off the 2nd summation on the rhs, we need to first explicitly establish that $e < 2^{t-1} - 1$.

In [ ]:
e_upper_bound = e_conditions.derive_element_upper_bound()

In [ ]:
e_upper_bound_plus_one = Less(e_upper_bound.rhs, Add(e_upper_bound.rhs, one)).prove()

In [ ]:
e_new_upper_bound = e_upper_bound.apply_transitivity(e_upper_bound_plus_one)

In [ ]:
e_new_upper_bound_simp = e_new_upper_bound.inner_expr().rhs.simplify()

### Now we're ready to partion off the first element of the second summation.

Nielsen & Chuang originally split off the first element of the _first_ summation, eventually leading to a bound requiring $e \ge 2$. Instead, we split off the first term of the _second_ summation, allowing us to eventually derive a bound valid for $e \ge 1$.

In [ ]:
temp_split_01 = _failure_upper_bound_lemma_inst_bound_05.rhs.operands[1].operands[1].partition_first(
    assumptions=defaults.assumptions+(second_sum_conditions,))

And use that to substitute into the larger inequality, with some auto-simplification combining like summation terms for us:

In [ ]:
# recall
_failure_upper_bound_lemma_inst_bound_05

In [ ]:
bound_07 = (
    _failure_upper_bound_lemma_inst_bound_05.inner_expr().rhs.
    operands[1].operands[1].substitute(temp_split_01))

In [ ]:
e_greater_zero = greater(e, zero).prove()

In [ ]:
# Checking on this one for omission; looks OK to OMIT
# InSet(frac(one, Exp(e, two)), RealPos).prove()
# one_over_e_sqrd_greater_zero = greater(frac(one, Exp(e, two)), zero).prove()

In [ ]:
bound_08 = bound_07.inner_expr().rhs.distribute(1)

In [ ]:
# summation-to-integral bounding theorem consistent with Nielsen & Chuang:
from proveit.numbers.summation import sum_integrate_ineq_NC
sum_integrate_ineq_NC

In [ ]:
from proveit.numbers.functions import one_over_x_sqrd_in_mon_dec_fxns
one_over_x_sqrd_in_mon_dec_fxns

In [ ]:
summand_in_mon_dec_funcs = one_over_x_sqrd_in_mon_dec_fxns.instantiate({x:l})

In [ ]:
# instantiate our summation-bounding integral theorem
from proveit import S
from proveit.numbers import RealPos
_S_sub, _f_sub, _a_sub, _b_sub, _x_sub = (
    RealPos, summand_in_mon_dec_funcs.element,
    bound_08.rhs.operands[0].operands[1].domain.lower_bound,
    bound_08.rhs.operands[0].operands[1].domain.upper_bound,
    l)
sum_integrate_ineq_NC_inst = sum_integrate_ineq_NC.instantiate(
    {S: _S_sub, f: _f_sub, a: _a_sub, b: _b_sub, x: _x_sub})

In [ ]:
InSet(Exp(l, two), Real).prove(
    assumptions=defaults.assumptions + (InSet(l, sum_integrate_ineq_NC_inst.rhs.domain),))

In [ ]:
InSet(l, sum_integrate_ineq_NC_inst.rhs.domain).derive_element_lower_bound(
    assumptions=defaults.assumptions + (InSet(l, sum_integrate_ineq_NC_inst.rhs.domain),))

In [ ]:
e_greater_eq_two = e_conditions.derive_element_lower_bound()

In [ ]:
# for the deduce_in_number_set for the integral in the next cell,
# we need to explicitly establish that ell ≠ 0
greater(l, zero).prove(
    assumptions=defaults.assumptions + (InSet(l, sum_integrate_ineq_NC_inst.rhs.domain),))

In [ ]:
sum_integrate_ineq_NC_inst.rhs.deduce_in_number_set(
    Real, assumptions=defaults.assumptions + (InSet(l, sum_integrate_ineq_NC_inst.rhs.domain),))

In [ ]:
bound_10 = bound_08.rhs.deduce_bound(sum_integrate_ineq_NC_inst,
            assumptions=defaults.assumptions + (InSet(l, sum_integrate_ineq_NC_inst.rhs.domain),))

In [ ]:
bound_11 = bound_08.apply_transitivity(bound_10)

In [ ]:
from proveit.numbers.integration import boundedInvSqrdIntegral
boundedInvSqrdIntegral

Need to establish that $2^{t-1}-1$ is in RealPos.

In [ ]:
# recall from earlier in the notebook:
e_new_upper_bound_simp

In [ ]:
integral_domain = bound_11.rhs.operands[0].operands[1].domain
_a_sub = integral_domain.lower_bound
_b_sub = integral_domain.upper_bound
integral_bound = boundedInvSqrdIntegral.instantiate({a: _a_sub, b: _b_sub})

In [ ]:
bound_12 = bound_11.rhs.deduce_bound(integral_bound)

In [ ]:
bound_13 = bound_11.apply_transitivity(bound_12)

In [ ]:
# qed fails here, with the following error:
# Sum simplification only implemented for a summation over an integer
# Interval of one instance variable where the upper and lower bounds are the same.
# %qed

In [ ]:
# then we need this before using transitivity futher below
from proveit.physics.quantum.QPE import _pfail_in_real
_pfail_in_real

In [ ]:
# fail_ineq_lemma_inst
_failure_upper_bound_lemma_inst

In [ ]:
# bound_14 = fail_ineq_lemma_inst.apply_transitivity(bound_13)
bound_14 = _failure_upper_bound_lemma_inst.apply_transitivity(bound_13)

In [ ]:
%qed